In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy
import os
import statsmodels.api as sm
import scipy.stats as stats
import statsmodels.formula.api as smf

if os.getcwd().split("\\")[-1] == "homework":
    os.chdir("../")
    
import cmds.custom_portfolio_management_helper as cpm
import cmds.portfolio_management_helper as pm

# Homework \#8

# Long-Term Capital Management, L.P. (A) [HBS 9-200-007]

Discuss these questions briefly, based on the info in the case. No need to quantitatively answer these questions.

## 1.1 Describe LTCM's investment strategy 
with regard to the following aspects:
- **Securities traded**: Fixed Income, low credit exposure.
- **Trading frequency** 1m-1y risk outlook, 
- **Skewness (Do they seek many small wins or a few big hits?)** Small wins
- **Forecasting (What is behind their selection of trades?)** Saught mispricing in spreads

## 1.2 What are LTCM's biggest advantages over its competitors?

Its ability to predict mispricings due to forieign macro economic policies
 


## 1.3 The case discusses four types of funding risk facing LTCM:
The case discusses specific ways in which LTCM manages each of these risks. Briefly discuss them.

- **Collateral Haircuts**: Not a focus since they rely on good financing terms from dealers due to LTCM's large capital base, low risk of assets, and mark-to-market transparency. Stress tested across asset classes.
- **Repo Maturity**: Here they are worried about securing long-term repos in bad conditions. They manage this by managing the overall repo maturity.
- **Equity Redemption**: Investor withdraws and margin calls both pay out of the funds cash. The firm runs liquidity shocks and projects future outgoing cashflows. 
- **Loan Access**: They have a loans from many banks which have are basically all or nothing. The loans terms dont get worse if the fund does bad, but if the NAV falls 50%, then they lose access.




## 1.4 
LTCM is largely in the business of selling liquidity and volatility. Describe how LTCM accounts for liquidity risk in their quantitative measurements.

They employed a process called **two-way mark to market**. The model security correlations at the 1 month adn 1 year.

## 1.5 Is leverage risk currently a concern for LTCM?
No it wasn't. They held many assets long and many assets short, these were often strongly correlated with each other. This results in a not-so-leveraged book.

## 1.6 
Many strategies of LTCM rely on converging spreads. LTCM feels that these are almost win/win situations because of the fact that if the spread converges, they make money. If it diverges, the trade becomes even more attractive, as convergence is still expected at a future date. What is the risk in these convergence trades?

Parallel movements.

# 2 LTCM Risk Decomposition
- On Canvas, find the data file, "ltcm_exhibits_data.xlsx". Get the gross and net (total) returns of LTCM from "Exhibit 2".
- Get the returns on SPY as well as the risk-free rate from the file, "gmo_analysis_data".

In [9]:
dfs_raw = pd.read_excel(r"data/ltcm_exhibits_data.xlsx" , sheet_name=None, skiprows=2)
dfs_raw.update(pd.read_excel(r"data/gmo_analysis_data.xlsx", sheet_name=None))
for key in dfs_raw.keys():
    print(f"{key}: {dfs_raw[key].shape}")


# display(dfs_raw['descriptions'].head())
annual_factor = 12

# ticker_mapping = {tick: name 
#                   for tick, name in zip(dfs_raw['descriptions'].iloc[:, 0], 
#                                                 dfs_raw['descriptions'].iloc[:, 1])}

# ticker_mapping


Copyright: (2, 1)
Exhibit 2: (58, 5)
Exhibit 4: (52, 5)
info: (7, 2)
signals: (335, 4)
risk-free rate: (418, 2)
total returns: (381, 4)


In [62]:
df_returns = (dfs_raw["Exhibit 2"]
              .rename(columns={dfs_raw["Exhibit 2"].columns[0]: "date",
                               "Fund Capital ($billions)": "aum",
                               "Gross Monthly Performancea": "return_gross",
                               "Net Monthly Performanceb": "return_net",
                       })
              .dropna()
              .set_index("date")
              )
df_returns = df_returns
df_returns.head()

,aum,return_gross,return_net,Index of Net Performance
date,,,,
1994-03-01,1.1000,-0.0110,-0.0130,0.9900
1994-04-01,1.1000,0.0140,0.0080,1.0000
1994-05-01,1.2000,0.0680,0.0530,1.0500
1994-06-01,1.2000,-0.0390,-0.0290,1.0200
1994-07-01,1.4000,0.1160,0.0840,1.1000


In [70]:
df_benchmarks = (dfs_raw["total returns"]
                 [["date", "SPY"]]
                 .merge(dfs_raw["risk-free rate"],
                        how="outer",
                        left_on="date",
                        right_on="date"
                        )
                .assign(date=lambda x: x["date"] + pd.DateOffset(1))  # Add 1 day
                .set_index("date")
                .interpolate() # Miss 1 risk free rate
                 )

print(df_benchmarks.isna().sum())
df_benchmarks.head()

SPY         38
TBill 3M     0
dtype: int64


,SPY,TBill 3M
date,,
1990-01-01,NaN,0.0764
1990-02-01,NaN,0.0774
1990-03-01,NaN,0.0790
1990-04-01,NaN,0.0777
1990-05-01,NaN,0.0774


In [71]:
# Merge

# Add market cols
df_merged = (df_returns
             .merge(df_benchmarks, how="left", left_index=True, right_index=True)
             .dropna() # dropped 3 years
             )

df_merged.head()

,aum,return_gross,return_net,Index of Net Performance,SPY,TBill 3M
date,,,,,,
1994-03-01,1.1000,-0.0110,-0.0130,0.9900,-0.0292,0.0350
1994-04-01,1.1000,0.0140,0.0080,1.0000,-0.0419,0.0368
1994-05-01,1.2000,0.0680,0.0530,1.0500,0.0112,0.0414
1994-06-01,1.2000,-0.0390,-0.0290,1.0200,0.0159,0.0414
1994-07-01,1.4000,0.1160,0.0840,1.1000,-0.0229,0.0433


## 2.1. Summary stats.
1. For both the gross and net series of LTCM excess returns, report the mean, volatility, and Sharpe ratios. (Annualize them.)
2. Report the skewness, kurtosis, and (historic) VaR(.05).
3. Comment on how these stats compare to SPY and other assets we have seen. How much do they differ between gross and net?


In [73]:
summary_cols = ["mean", "vol", "sharpe", "skewness", "kurtosis", "5% VaR"]

cpm.calc_summary_stats(df_returns[["return_gross", "return_net"]],
                       annual_factor=annual_factor,
                       summary_cols=summary_cols
                       ).T

,mean,vol,sharpe,kurtosis,skewness,5% VaR
return_gross,0.2939,0.1364,2.1553,1.5694,-0.2964,-0.0264
return_net,0.2072,0.1119,1.8513,2.9055,-0.8179,-0.0224


These have a larger mean return then we have seen before, good sharp ratios too. The gross return is about 10% more then net return.

## 2.2. Using the series of net LTCM excess returns, denoted $\tilde{r}^{\mathrm{LTCM}}$, estimate the following regression:
$$\tilde{r}_{t}^{\mathrm{LTCM}}=\alpha+\beta^{m} \tilde{r}_{t}^{m}+\epsilon_{t}$$

1. Report $\alpha$ and $\beta^{m}$. Report the $R^{2}$ stat.
2. From this regression, does LTCM appear to be a "closet indexer"?
3. From the regression, does LTCM appear to deliver excess returns beyond the risk premium we expect from market exposure?


In [ ]:
def factor_model(df, y_var, x_vars, intercept=True, lag=0):
    
    if not isinstance(x_vars, list):
        x_vars = [x_vars]
    # Run regression
    formula = f"{y_var} ~ {' + '.join(x_vars)}"
    if not intercept:
        formula = formula + " - 1"
    model = smf.ols(formula=formula, data=df)
    results = model.fit()
    summary = results.summary()

    return model, results

def risk_summary(df, market_col, index_cols=None, annual_scale=12):
    if index_cols is None:
        index_cols = list(df.columns)
        if market_col in index_cols: index_cols.remove(market_col)


    data = []
    for c in index_cols:
        _, results = factor_model(df, c, market_col)
        market_beta = results.params[market_col]

        mu_return = df[c].mean() * annual_scale
        vol_return = df[c].std() * np.sqrt(annual_scale)

        data.append({
            "index": c,
            "market_alpha": results.params["Intercept"],
            "market_beta": market_beta,
            "trenor_ratio": mu_return / market_beta,
            "info_ratio": mu_return / vol_return,
            "R2": results.rsquared,
        })
    return pd.DataFrame(data).set_index("index")


main_col = "return_net"
market_col = "SPY"
risk_summary(df_merged[[main_col, market_col]], market_col)


,market_alpha,market_beta,trenor_ratio,info_ratio,R2
index,,,,,
return_net,0.0197,0.0389,6.3154,2.4663,0.0018


These returns have low beta to the market and a decent monthly alpha. So no its not a closet indexer, and yes it appears to be delivering excess returns beyond market risk premium.


## 2.3. Let's check for non-linear market exposure. Run the following regression on LTCM's net excess returns:
$$\tilde{r}_{t}^{\mathrm{LTCM}}=\alpha+\beta_{1} \tilde{r}_{t}^{m}+\beta_{2}\left(\tilde{r}_{t}^{m}\right)^{2}+\epsilon_{t}$$

1. Report $\beta_{1}, \beta_{2}$, and the $R^{2}$ stat.
2. Does the quadratic market factor do much to increase the overall LTCM variation explained by the market?
3. From the regression evidence, does LTCM's market exposure behave as if it is long market options or short market options?
4. Should we describe LTCM as being positively or negatively exposed to market volatility?


In [87]:
df_reg = df_merged.copy()
df_reg["SPY2"] = df_reg["SPY"] ** 2


data = []
_, results = factor_model(df_reg, "return_net", ["SPY", "SPY2"])

pd.DataFrame([{
    "alpha": results.params["Intercept"],
    "beta_1": results.params["SPY"],
    "beta_2": results.params["SPY2"],
    "R2": results.rsquared,
}])

,alpha,beta_1,beta_2,R2
0,0.0244,0.1672,-5.4312,0.0497


There is improvement in the R squared but nothing crazy. This suggests the fund is short market options and is negatively exposed to market vol.

## 2.4. Let's try to pinpoint the nature of LTCM's nonlinear exposure. Does it come more from exposure to up-markets or down-markets? Run the following regression on LTCM's net excess returns:

$$
\tilde{r}_{t}^{\mathrm{LTCM}}=\alpha+\beta \tilde{r}_{t}^{m}+\beta_{u} \max \left(\tilde{r}_{t}^{m}-k_{1}, 0\right)+\beta_{d} \max \left(k_{2}-\tilde{r}_{t}^{m}, 0\right)+\epsilon_{t}
$$

where $k_{1}=.03$ and $k_{2}=-.03$. (This is roughly one standard deviation of $\tilde{r}^{m}$.)
1. Report $\beta, \beta_{u}, \beta_{d}$, and the $R^{2}$ stat.
2. Is LTCM long or short the call-like factor? And the put-like factor?
3. Which factor moves LTCM more, the call-like factor, or the put-like factor?
4. In the previous problem, you commented on whether LTCM is positively or negatively exposed to market volatility. Using this current regression, does this volatility exposure come more from being long the market's upside? Short the market's downside? Something else?


In [89]:
k1 = .03
k2 = -.03
df_reg = df_merged.copy()
df_reg["SPY_u"] = np.maximum(df_reg["SPY"] - k1, 0)
df_reg["SPY_d"] = np.minimum(k2 - df_reg["SPY"], 0)


data = []
_, results = factor_model(df_reg, "return_net", ["SPY", "SPY_u", "SPY_d"])

pd.DataFrame([{
    "alpha": results.params["Intercept"],
    "beta": results.params["SPY"],
    "beta_u": results.params["SPY_u"],
    "beta_d": results.params["SPY_d"],
    "R2": results.rsquared,
}])

,alpha,beta,beta_u,beta_d,R2
0,0.0604,1.3115,-0.4804,1.2286,0.0548


Short the call and long the put like factors. The put factor moves the LTCM more. From this regression the negative market vol exposure comes from being long the downside.


# 3 The FX Carry Trade

## Data
Find an Excel data file, "data/fx_rf_data.xlsx". The file has two sets of data:

- "risk-free rates" - daily quotes of annualized risk-free rates across currencies.
- "exchange rates" - daily quotes of spot FX rates expressed as direct quotes to the USD.


In [132]:
dfs_raw = pd.read_excel(r"data/fx_rf_data.xlsx" , sheet_name=None)
for key in dfs_raw.keys():
    print(f"{key}: {dfs_raw[key].shape}")
    display(dfs_raw[key].head())

annual_factor = 1

risk-free rates: (4393, 7)


,date,USD,JPY,EUR,GBP,MXN,CHF
0,2008-01-01,0.0440,0.0048,0.0425,0.0570,0.0750,0.0206
1,2008-01-02,0.0416,0.0048,0.0360,0.0547,0.0750,0.0206
2,2008-01-03,0.0435,0.0048,0.0394,0.0560,0.0750,0.0215
3,2008-01-04,0.0422,0.0048,0.0407,0.0550,0.0750,0.0209
4,2008-01-07,0.0422,0.0049,0.0410,0.0546,0.0750,0.0202


exchange rates: (4393, 6)


,date,JPY,EUR,GBP,MXN,CHF
0,2008-01-01,0.0090,1.4592,1.9864,0.0918,0.8824
1,2008-01-02,0.0091,1.4715,1.9808,0.0916,0.8933
2,2008-01-03,0.0091,1.4750,1.9710,0.0919,0.9002
3,2008-01-04,0.0092,1.4743,1.9740,0.0915,0.9022
4,2008-01-07,0.0092,1.4696,1.9704,0.0918,0.8957



### Data Processing
- For risk-free rate data, $r_{t, t+1}^{f, i}$, the rate is known and reported in the data at time $t$. Thus, at any given date $t$, the data file is reporting both $S_{t}^{i}$ and $r_{t, t+1}^{f, i}$.
- The theory says to use log risk-free rates. You have the risk-free rate in levels: use the following equation to convert them:

$$r_{t, t+1}^{f, i}=\ln \left(1+r_{t, t+1}^{f, i}\right)$$

- The theory says to use log spot FX prices. You have the FX prices in levels, so directly take their logarithms:

$$s_{t}^{i}=\ln \left(S_{t}^{i}\right)$$


In [143]:
# Risk free rates
df_rfs = (dfs_raw["risk-free rates"].copy()
          .set_index("date")
          .apply(lambda x: np.log(1 + x))
          )
display(df_rfs.head())

# Spot Rates
df_spot = (dfs_raw["exchange rates"].copy()
           .set_index("date")
           .apply(lambda x: np.log(x))
           )
display(df_spot.head())


,USD,JPY,EUR,GBP,MXN,CHF
date,,,,,,
2008-01-01,0.0431,0.0048,0.0416,0.0554,0.0723,0.0204
2008-01-02,0.0407,0.0048,0.0354,0.0533,0.0723,0.0204
2008-01-03,0.0425,0.0048,0.0386,0.0545,0.0723,0.0213
2008-01-04,0.0413,0.0048,0.0398,0.0535,0.0723,0.0207
2008-01-07,0.0413,0.0049,0.0402,0.0532,0.0723,0.0200


,JPY,EUR,GBP,MXN,CHF
date,,,,,
2008-01-01,-4.7153,0.3779,0.6863,-2.3878,-0.1251
2008-01-02,-4.6974,0.3863,0.6835,-2.3901,-0.1128
2008-01-03,-4.6942,0.3887,0.6785,-2.3874,-0.1051
2008-01-04,-4.6876,0.3882,0.6801,-2.3910,-0.1029
2008-01-07,-4.6930,0.3850,0.6782,-2.3886,-0.1101



## 3.1 The Static Carry Trade
Define the log return of holding the foreign currency using log values of the risk-free rate and log values of the FX rates:
$$\mathrm{r}_{t+1}^{i} \equiv s_{t+1}^{i}-s_{t}^{i}+\mathrm{r}_{t, t+1}^{f, i}$$

Then the excess log return relative to USD, is expressed as
$$\tilde{\mathrm{r}}_{t+1}^{i} \equiv s_{t+1}^{i}-s_{t}^{i}+\mathrm{r}_{t, t+1}^{f, i}-\mathrm{r}_{t, t+1}^{f, \$}$$


In [134]:
df_carry = pd.DataFrame(index = df_spot.index, columns = df_spot.columns)
for c in df_spot.columns:
    df_carry[c] = df_spot[c] - df_spot[c].shift(1) + df_rfs[c].shift(1)
df_carry = df_carry.dropna()
display(df_carry.head())

df_carry_usd = pd.DataFrame(index = df_spot.index, columns = df_spot.columns)
for c in df_spot.columns:
    df_carry_usd[c] = df_carry[c] - df_rfs["USD"]
df_carry_usd = df_carry_usd.dropna()
display(df_carry_usd.head())

,JPY,EUR,GBP,MXN,CHF
date,,,,,
2008-01-02,0.0227,0.0500,0.0526,0.0700,0.0327
2008-01-03,0.0079,0.0377,0.0483,0.0750,0.0281
2008-01-04,0.0114,0.0382,0.0560,0.0687,0.0235
2008-01-07,-0.0007,0.0367,0.0517,0.0747,0.0135
2008-01-08,0.0075,0.0409,0.0546,0.0696,0.0236


,JPY,EUR,GBP,MXN,CHF
date,,,,,
2008-01-02,-0.0180,0.0093,0.0119,0.0293,-0.0080
2008-01-03,-0.0346,-0.0048,0.0058,0.0325,-0.0144
2008-01-04,-0.0299,-0.0032,0.0147,0.0274,-0.0178
2008-01-07,-0.0420,-0.0047,0.0104,0.0334,-0.0278
2008-01-08,-0.0350,-0.0016,0.0121,0.0271,-0.0190



For each foreign currency, $i$, calculate the excess log return series, $\tilde{r}_{t+1}$. Report the following stats, (based on the excess log returns.) Annualize them.

1. mean
2. volatility
3. Sharpe ratio

What differences do you see across currencies?


In [146]:
summary_cols = ["mean", "vol", "sharpe"]

cpm.calc_summary_stats(df_carry_usd,
                       annual_factor=annual_factor,
                       summary_cols=summary_cols
                       ).T

,mean,vol,sharpe
JPY,-0.0117,0.0170,-0.6864
EUR,-0.0068,0.0125,-0.5417
GBP,-0.0002,0.0103,-0.0167
MXN,0.0460,0.0133,3.4555
CHF,-0.0128,0.0146,-0.8799


These means are all close to 0 with small vols. The sharpe ratios are all under a magnitude of 1 except for MXN.

### 3.2 Implications for UIP
1. Do any of these stats contradict the (log version) of Uncovered Interest Parity (UIP)?
2. A long position in which foreign currency offered the best Sharpe ratio over the sample?
3. Are there any foreign currencies for which a long position earned a negative excess return (in USD) over the sample?

todo

## 3.3 Predicting FX

For each foreign currency, test whether interest-rate differentials can predict growth in the foreignexchange rate. ${ }^{1}$ Do this by estimating the following forecasting regression::

$$
s_{t+1}^{i}-s_{t}^{i}=\alpha^{i}+\beta^{i}\left(\mathrm{r}_{t, t+1}^{f, \$}-\mathrm{r}_{t, t+1}^{f, i}\right)+\epsilon_{t+1}^{i}
$$

where $\mathrm{r}^{f, i}$ denotes the risk-free rate of currency $i$, and $s^{i}$ denotes the FX rate for currency $i$. Again, note that both $\mathrm{r}_{t, t+1}^{f, \$}$ and $s_{t}$ are determined at time $t$.

1. Make a table with columns corresponding to a different currency regression. Report the regression estimates $\alpha^{i}$ and $\beta^{i}$ in the first two rows. Report the $R^{2}$ stat in the third row.


In [151]:

data = []
for c in df_carry_usd.columns:
    # s_{t+1}^{i}-s_{t}^{i}=\alpha^{i}+\beta^{i}\left(\mathrm{r}_{t, t+1}^{f, \$}-\mathrm{r}_{t, t+1}^{f, i}\right)+\epsilon_{t+1}^{i}
    df_reg = pd.DataFrame()
    df_reg["y"] = df_spot[c] - df_spot[c].shift(1)
    df_reg["x"] = df_rfs["USD"] - df_rfs[c]
    
    _, results = factor_model(df_reg.dropna(), "y", "x")
    data.append({
        "index": c,
        "alpha": results.params["Intercept"],
        "beta": results.params["x"],
        "R2": results.rsquared,
    })
df_reg_results = pd.DataFrame(data).set_index("index").T
df_reg_results

index,JPY,EUR,GBP,MXN,CHF
alpha,-0.0001,-0.0001,-0.0001,-0.0002,0.0000
beta,0.0016,0.0092,0.0248,-0.0012,0.0042
R2,0.0000,0.0003,0.0012,0.0000,0.0001



2. Suppose the foreign risk-free rate increases relative to the US rate.
    1. For which foreign currencies would we predict a relative strengthening of the USD in the following period?
    2. For which currencies would we predict relative weakening of the USD in the following period?
    3. This FX predictability is strongest in the case of which foreign currency?

todo


## 3.4 The Dynamic Carry Trade
Use this to write $\mathbb{E}_{t}\left[\tilde{\mathrm{r}}_{t+1}^{i}\right]$ as a function of the interest-rate differential as well as $\alpha$ and $\beta$ from this FX regression.
$$\mathbb{E}_{t}\left[s_{t+1}-s_{t}\right]=\alpha+\beta\left(\mathrm{r}_{t, t+1}^{f, \oiint}-\mathrm{r}_{t, t+1}^{f, i}\right)$$

Then use the definition of excess (log) returns on FX:
$$\tilde{\mathrm{r}}_{t+1}^{i}=s_{t+1}-s_{t}-\left(\mathrm{r}_{t, t+1}^{f, \oiint}-\mathrm{r}_{t, t+1}^{f, i}\right)$$

Rearranging, this implies the following forecast for excess log returns:

$$
\mathbb{E}_{t}\left[\tilde{\mathrm{r}}_{t+1}^{i}\right]=\alpha+(\beta-1)\left(\mathrm{r}_{t, t+1}^{f, \mathbb{S}}-\mathrm{r}_{t, t+1}^{f, i}\right)
$$

1. Use your regression estimates from Problem 3 along with the formula above to calculate the fraction of months for which the estimated FX risk premium positive. That is, for each $i$, calculate how often in the time-series we have
$$\mathbb{E}_{t}\left[\tilde{\mathrm{r}}_{t+1}^{i}\right]>0$$

In [159]:
data = []
for c in df_carry_usd.columns:
    alpha = df_reg_results.loc["alpha", c]
    beta = df_reg_results.loc["beta", c]
    
    
    data.append({
        "index": c,
        "estimated": np.mean(alpha + (beta - 1) * (df_rfs["USD"] - df_rfs[c])),
        "actual": np.mean(df_carry_usd[c])
    })
pd.DataFrame(data).set_index("index").T

index,JPY,EUR,GBP,MXN,CHF
estimated,-0.0117,-0.0068,-0.0002,0.0460,-0.0129
actual,-0.0117,-0.0068,-0.0002,0.0460,-0.0128



2. Which currencies most consistently have a positive FX risk premium? And for which currencies does the FX risk premium most often go negative?

todo

3. Explain how we could use these conditional risk premia to improve the static carry trade returns calculated in Problem 1.

todo